In [4]:
!pip install catboost

In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

kaggle/input\avjanata-hackhealth-care-analytics2\test.csv
kaggle/input\avjanata-hackhealth-care-analytics2\train.csv
kaggle/input\avjanata-hackhealth-care-analytics2\train_data_dict.csv


In [14]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold,KFold,GroupKFold
from sklearn.metrics import accuracy_score
pd.set_option('display.max_columns', 100)
import warnings
warnings.filterwarnings("ignore")
import time

In [16]:
train=pd.read_csv('kaggle/input/avjanata-hackhealth-care-analytics2/train.csv')
test=pd.read_csv('kaggle/input/avjanata-hackhealth-care-analytics2/test.csv')
sub=pd.read_csv('kaggle/input/avjanata-hackhealth-care-analytics2/sample_submission_lfbv3c3.csv')
des=pd.read_csv('kaggle/input/avjanata-hackhealth-care-analytics2/train_data_dict.csv')

### Concating

In [17]:
train['train_or_test']='train'
test['train_or_test']='test'
df=pd.concat([train,test])

### Mapping

In [18]:
hospital_type_map={
 'a': 0,
 'b': 1,
 'c': 2,
 'e': 3,
 'd': 4,
 'f': 5,
 'g': 6} 

hospital_region_map = {'X': 0, 'Y': 1, 'Z': 2}

dep_map={'gynecology': 0,
 'anesthesia': 1,
 'radiotherapy': 2,
 'TB & Chest disease': 3,
 'surgery': 4}

ward_type_map ={'R': 0, 'Q': 1, 'S': 2, 'P': 3, 'T': 4, 'U': 5}
ward_fac_map ={'F':0, 'E': 1, 'D':2, 'C': 3, 'B': 4, 'A': 5}
admiss_map = {'Trauma': 0, 'Emergency': 1, 'Urgent': 2}
stay_map = {'21-30': 2,
 '11-20': 1,
 '31-40': 3,
 '51-60': 5,
 '0-10': 0,
 '41-50': 4,
 '71-80': 7,
 'More than 100 Days': 10,
 '81-90': 8,
 '91-100': 9,
 '61-70': 6}

In [19]:
df['Hospital_type_code'] = df['Hospital_type_code'].map(hospital_type_map)
df['Hospital_region_code'] = df['Hospital_region_code'].map(hospital_region_map)
df['Department'] = df['Department'].map(dep_map)
df['Ward_Type'] = df['Ward_Type'].map(ward_type_map)
df['Ward_Facility_Code'] = df['Ward_Facility_Code'].map(ward_fac_map)
df['Type of Admission'] = df['Type of Admission'].map(admiss_map)
df['Stay'] = df['Stay'].map(stay_map)

In [20]:
df['Severity of Illness'] = df['Severity of Illness'].map({'Minor': 1,
                          'Moderate ': 2,
                          'Extreme': 3})

In [21]:
df['Age'] = df['Age'].map({'0-10': 5,
                          '11-20': 15,
                          '21-30': 25,
                          '31-40': 35,
                          '41-50': 45,
                          '51-60': 55,
                          '61-70': 65,
                          '71-80': 75,
                          '81-90': 85,
                          '91-100': 95
                         })

### Feature Engineering

In [22]:
df['mean_Admission_Deposit_per_illness']=df.groupby(['Severity of Illness'])['Admission_Deposit'].transform('mean')
df['mean_Admission_Deposit_per_admission']=df.groupby(['Type of Admission'])['Admission_Deposit'].transform('mean')
df['mean_Admission_Deposit_per_bed']=df.groupby(['Bed Grade'])['Admission_Deposit'].transform('mean')
df['mean_Admission_Deposit_per_department']=df.groupby(['Department'])['Admission_Deposit'].transform('mean')
df['mean_Admission_Deposit_per_hosp']=df.groupby(['Hospital_code'])['Admission_Deposit'].transform('mean')

In [23]:
df['sum_Admission_Deposit_per_illness']=df.groupby(['Severity of Illness'])['Admission_Deposit'].transform('sum')
df['sum_Admission_Deposit_per_admission']=df.groupby(['Type of Admission'])['Admission_Deposit'].transform('sum')
df['sum_Admission_Deposit_per_bed']=df.groupby(['Bed Grade'])['Admission_Deposit'].transform('sum')
df['sum_Admission_Deposit_per_department']=df.groupby(['Department'])['Admission_Deposit'].transform('sum')
df['sum_Admission_Deposit_per_hosp']=df.groupby(['Hospital_code'])['Admission_Deposit'].transform('sum')

In [24]:
df['max_Admission_Deposit_per_illness']=df.groupby(['Severity of Illness'])['Admission_Deposit'].transform('max')
df['max_Admission_Deposit_per_admission']=df.groupby(['Type of Admission'])['Admission_Deposit'].transform('max')
df['max_Admission_Deposit_per_bed']=df.groupby(['Bed Grade'])['Admission_Deposit'].transform('max')
df['max_Admission_Deposit_per_department']=df.groupby(['Department'])['Admission_Deposit'].transform('max')
df['max_Admission_Deposit_per_hosp']=df.groupby(['Hospital_code'])['Admission_Deposit'].transform('max')

In [25]:
df['min_Admission_Deposit_per_illness']=df.groupby(['Severity of Illness'])['Admission_Deposit'].transform('min')
df['min_Admission_Deposit_per_admission']=df.groupby(['Type of Admission'])['Admission_Deposit'].transform('min')
df['min_Admission_Deposit_per_bed']=df.groupby(['Bed Grade'])['Admission_Deposit'].transform('min')
df['min_Admission_Deposit_per_department']=df.groupby(['Department'])['Admission_Deposit'].transform('min')
df['min_Admission_Deposit_per_hosp']=df.groupby(['Hospital_code'])['Admission_Deposit'].transform('min')

In [26]:
df['mean_Admission_Deposit_per_patient_hosp']=df.groupby(['patientid','Hospital_code'])['Admission_Deposit'].transform('mean')
df['sum_Admission_Deposit_per_patient_hosp']=df.groupby(['patientid','Hospital_code'])['Admission_Deposit'].transform('sum')
df['max_Admission_Deposit_per_patient_hosp']=df.groupby(['patientid','Hospital_code'])['Admission_Deposit'].transform('max')
df['min_Admission_Deposit_per_patient_hosp']=df.groupby(['patientid','Hospital_code'])['Admission_Deposit'].transform('min')

In [27]:
df['illness_count']=df.groupby('Hospital_code')['Severity of Illness'].transform('count')
df['bed_count']=df.groupby('Hospital_code')['Bed Grade'].transform('count')
df['room_count']=df.groupby('Hospital_code')['Available Extra Rooms in Hospital'].transform('count')
df['Department_count']=df.groupby('Hospital_code')['Department'].transform('count')
df['ward_count']=df.groupby('Hospital_code')['Ward_Type'].transform('count')
df['TOA_count']=df.groupby('Hospital_code')['Type of Admission'].transform('count')

In [28]:
df['meanVisitors with Patient_per_patient']=df.groupby(['patientid'])['Visitors with Patient'].transform('mean')
df['meanVisitors with Patient_per_patient']=df.groupby(['patientid'])['Visitors with Patient'].transform('sum')
df['meanVisitors with Patient_per_patient']=df.groupby(['patientid'])['Visitors with Patient'].transform('max')
df['meanVisitors with Patient_per_patient']=df.groupby(['patientid'])['Visitors with Patient'].transform('min')

In [29]:
df['unique_hospital_visited']=df.groupby('patientid')['Hospital_code'].transform('nunique')
df['unique_hospitaltype_visited']=df.groupby('patientid')['Hospital_type_code'].transform('nunique')
df['count_visited_same_hospital']=df.groupby(['patientid','Hospital_code'])['Hospital_code'].transform('count')

df['mean_Admission_Deposit_per_patient']=df.groupby(['patientid'])['Admission_Deposit'].transform('mean')
df['sum_Admission_Deposit_per_patient']=df.groupby(['patientid'])['Admission_Deposit'].transform('sum')
df['max_Admission_Deposit_per_patient']=df.groupby(['patientid'])['Admission_Deposit'].transform('max')
df['min_Admission_Deposit_per_patient']=df.groupby(['patientid'])['Admission_Deposit'].transform('min')

df['mean_visitors_per_patient']=df.groupby(['patientid'])['Visitors with Patient'].transform('mean')
df['unique city']=df.groupby('patientid')['City_Code_Hospital'].transform('nunique')
df['unique illness type']=df.groupby('patientid')['Severity of Illness'].transform('nunique')
df['unique admission type']=df.groupby('patientid')['Type of Admission'].transform('nunique')

df['mean_Admission_Deposit_per_patient_in_same_hospital']=df.groupby(['patientid','Hospital_code'])['Admission_Deposit'].transform('mean')

In [30]:
df['total_no_of_patients_in_hospital']=df.groupby('Hospital_code')['patientid'].transform('count')
df['total_no_of_patients_in_hospital_from_same_city']=df.groupby(['Hospital_code','City_Code_Patient'])['patientid'].transform('count')
df['total_no_of_patients_in_patientcity']=df.groupby('City_Code_Patient')['patientid'].transform('count')
df['total_no_of_patients_in_hospitalcity']=df.groupby('City_Code_Hospital')['patientid'].transform('count')

In [31]:
df['number_of_time_patient_visited']=df.groupby('patientid')['patientid'].transform('count')
df['Total_Hospitals_in_city']=df.groupby('City_Code_Hospital')['Hospital_code'].transform('nunique')
df['Total_Hospitals_in_city_of_same_type']=df.groupby(['City_Code_Hospital','Hospital_type_code'])['Hospital_code'].transform('nunique')

In [32]:
df['total_no_of_patients_per_Department']=df.groupby('Department')['patientid'].transform('count')
df['total_no_of_patients_per_Severity of Illness']=df.groupby('Severity of Illness')['patientid'].transform('count')
df['Total_Hospitals_in_region']=df.groupby('Hospital_region_code')['patientid'].transform('count')
df['total_no_of_patients_in_hospitalward']=df.groupby(['Hospital_code','Ward_Type'])['patientid'].transform('count')
df['total_no_of_patients_in_hospitaldepartment']=df.groupby(['Hospital_code','Department'])['patientid'].transform('count')
df['total_no_of_patients_in_hospitalBedGrade']=df.groupby(['Hospital_code','Bed Grade'])['patientid'].transform('count')
df['total_no_of_patients_in_hospitalSeverityofIllness']=df.groupby(['Hospital_code','Type of Admission'])['patientid'].transform('count')
df['unique_patientcity_in_hospital']=df.groupby('Hospital_code')['City_Code_Patient'].transform('nunique')
df['Average_deposit']=df.groupby('Hospital_code')['Available Extra Rooms in Hospital'].transform('mean')
df['Total_Hospitals_in_patientcity']=df.groupby('City_Code_Patient')['Hospital_code'].transform('nunique')
df['Total_Hospitals_in_region']=df.groupby('Hospital_region_code')['Hospital_code'].transform('nunique')

### Rank features

In [33]:
df["RANK"] = df.groupby("patientid")['patientid'].rank(method="first", ascending=True)
df["RANK_avg"] = df.groupby("patientid")['patientid'].rank(method="average", ascending=True)
df["RANK_max"] = df.groupby("patientid")['patientid'].rank(method="max", ascending=True)
df["RANK_min"] = df.groupby("patientid")['patientid'].rank(method="min", ascending=True)
df["RANK_DIFF"] = df['RANK_max'] - df['RANK_min']
df["RANK_Type_Admission"] = df.groupby(['patientid','Type of Admission'])['patientid'].rank(method='first',ascending=True)
df["RANK_Type_City_Code"] = df.groupby(['patientid','City_Code_Hospital'])['patientid'].rank(method='first',ascending=True)
df["RANK_Type_hosp_Code"] = df.groupby(['patientid','Hospital_code'])['patientid'].rank(method='first',ascending=True)
df["RANK_Type_hosp_type_Code"] = df.groupby(['patientid','Hospital_type_code'])['patientid'].rank(method='first',ascending=True)
df["RANK_Type_rooms"] = df.groupby(['patientid','Available Extra Rooms in Hospital'])['patientid'].rank(method='first',ascending=True)
df["RANK_Type_Department"] = df.groupby(['patientid','Department'])['patientid'].rank(method='first',ascending=True)
df["RANK_Type_Ward_Type"] = df.groupby(['patientid','Ward_Type'])['patientid'].rank(method='first',ascending=True)
df["RANK_Type_Ward_Facility_Code"] = df.groupby(['patientid','Ward_Facility_Code'])['patientid'].rank(method='first',ascending=True)
df["RANK_Type_Hospital_region_code"] = df.groupby(['patientid','Hospital_region_code'])['patientid'].rank(method='first',ascending=True)
df['Rank_City_Code_Patient'] = df.groupby(['patientid','City_Code_Patient'])['patientid'].rank(method='first',ascending=True)
df['Rank_City_Code_Vist'] = df.groupby(['patientid','Visitors with Patient'])['patientid'].rank(method='first',ascending=True)
df['Rank_ill+admiss'] = df.groupby(['patientid','Type of Admission','Severity of Illness'])['patientid'].rank(method='first',ascending=True)
df['Rank_ill+admiss_max'] = df.groupby(['patientid','Type of Admission','Severity of Illness'])['patientid'].rank(method='max',ascending=True)
df['Rank_ill+admiss_min'] = df.groupby(['patientid','Type of Admission','Severity of Illness'])['patientid'].rank(method='min',ascending=True)
df['Rank_ill+admiss_avg'] = df.groupby(['patientid','Type of Admission','Severity of Illness'])['patientid'].rank(method='average',ascending=True)
df['Rank_ill+admiss_diff'] = df['Rank_ill+admiss_max'] - df['Rank_ill+admiss_min']
cols_rank = ['Type of Admission','City_Code_Hospital','Hospital_code','Hospital_type_code','Available Extra Rooms in Hospital','Department','Ward_Type','Ward_Facility_Code',
             'Hospital_region_code','City_Code_Patient','Visitors with Patient']
for i in cols_rank:
    df[f"RANK_{i}_min"] = df.groupby(['patientid',i])['patientid'].rank(method='min',ascending=True)
    df[f"RANK_{i}_max"] = df.groupby(['patientid',i])['patientid'].rank(method='max',ascending=True)
    df[f"RANK_{i}_avg"] = df.groupby(['patientid',i])['patientid'].rank(method='average',ascending=True)
    df[f'diff_{i}_max_min'] = df[f"RANK_{i}_max"] -  df[f"RANK_{i}_min"]

In [34]:
train=df.loc[df.train_or_test.isin(['train'])]
test=df.loc[df.train_or_test.isin(['test'])]
train.drop(columns={'train_or_test'},axis=1,inplace=True)
test.drop(columns={'train_or_test'},axis=1,inplace=True)

In [35]:
x=train.drop(columns={'case_id','Stay'},axis=1)
y=train.loc[:,['Stay']]
test=test.drop(columns={'case_id','Stay'},axis=1)

### StratifiedKFOLD

In [36]:
%%time
err = [] 
y_pred_tot_lgm = np.zeros((len(test), 11))


fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=2020)
i = 1

for train_index, test_index in fold.split(x, y):
    x_train, x_val = x.iloc[train_index], x.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    m = CatBoostClassifier(n_estimators=10000,
                       random_state=2020,
                       eval_metric='Accuracy',
                       learning_rate=0.08,
                       depth=8,
                       bagging_temperature=0.3,
                       task_type='GPU'
                       #num_leaves=64
                       
                       )
    m.fit(x_train, y_train,eval_set=[(x_val, y_val)], early_stopping_rounds=100,verbose=200)
    pred_y = m.predict(x_val)
    print(i, " err_lgm: ", accuracy_score(y_val,pred_y))
    err.append(accuracy_score(y_val,pred_y))
    y_pred_tot_lgm+= m.predict_proba(test)
    i = i + 1
y_pred_tot_lgm=y_pred_tot_lgm/10
sum(err)/10

CatBoostError: c:/program files (x86)/go agent/pipelines/buildmaster/catboost.git/catboost/cuda/cuda_lib/cuda_base.h:281: CUDA error 35: CUDA driver version is insufficient for CUDA runtime version

In [37]:
44.1941

44.1941

In [38]:
stay_map = {2:'21-30',
 1:'11-20',
 3:'31-40',
 5:'51-60',
 0:'0-10',
 4:'41-50',
 7:'71-80',
 10:'More than 100 Days',
 8:'81-90',
 9:'91-100',
 6:'61-70'}

#### Predicting from average

In [39]:
sub['Stay'] = [np.argmax(x) for x in y_pred_tot_cat]
sub['Stay']=sub['Stay'].map(stay_map)
sub['Stay'].value_counts(normalize=True)
sub.to_csv('lgbm_final_data.csv',index=False)

NameError: name 'y_pred_tot_cat' is not defined